<a href="https://colab.research.google.com/github/v-aguiar/reconhecimento-facial/blob/main/Face_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reconhecimento facial usando OpenCV via Webcam

Esse projeto utiliza o modelo haarcascades da OpenCv para realizar a detecção facial.
Também é realizado um processo de captura de amostrar do rosto do usuário.

Após a captura de amostras e treinamento do modelo, o sistema é capaz de detectar rostos e entre "Desconhecido" ou "Nome_do_usuário"


In [67]:
# import dependencies
import cv2
import numpy as np
import os
import time
from IPython.display import display, Javascript, HTML
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import PIL
import io
import html
import matplotlib.pyplot as plt
%matplotlib inline

In [71]:
class FaceRecognitionSystem:
    def __init__(self):
        self.face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )
        self.recognizer = None
        self.labels = {}
        self.is_capturing = False
        self.should_stop = False

    def init_webcam(self):
        display(HTML('''
            <div id="webcam-container"></div>
            <button onclick="document.getElementById('webcam-container').innerHTML = ''">
                Parar Webcam
            </button>
        '''))

    def js_to_image(self, js_reply):
        image_bytes = b64decode(js_reply.split(',')[1])
        jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
        return cv2.imdecode(jpg_as_np, flags=1)

    def start_video_stream(self):
        js = Javascript('''
          var video;
          var div = null;
          var stream;
          var captureCanvas;
          var imgElement;
          var labelElement;

          var pendingResolve = null;
          var shutdown = false;

          function removeDom() {
            stream.getVideoTracks()[0].stop();
            video.remove();
            div.remove();
            video = null;
            div = null;
            stream = null;
            imgElement = null;
            captureCanvas = null;
            labelElement = null;
          }

          function onAnimationFrame() {
            if (!shutdown) {
              window.requestAnimationFrame(onAnimationFrame);
            }
            if (pendingResolve) {
              var result = "";
              if (!shutdown) {
                captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
                result = captureCanvas.toDataURL('image/jpeg', 0.8)
              }
              var lp = pendingResolve;
              pendingResolve = null;
              lp(result);
            }
          }

          async function createDom() {
            if (div !== null) {
              return stream;
            }

            div = document.createElement('div');
            div.style.border = '2px solid black';
            div.style.padding = '3px';
            div.style.width = '100%';
            div.style.maxWidth = '600px';
            document.body.appendChild(div);

            const modelOut = document.createElement('div');
            modelOut.innerHTML = "<span>Status:</span>";
            labelElement = document.createElement('span');
            labelElement.innerText = 'No data';
            labelElement.style.fontWeight = 'bold';
            modelOut.appendChild(labelElement);
            div.appendChild(modelOut);

            video = document.createElement('video');
            video.style.display = 'block';
            video.width = div.clientWidth - 6;
            video.setAttribute('playsinline', '');
            video.onclick = () => { shutdown = true; };
            stream = await navigator.mediaDevices.getUserMedia(
                {video: { facingMode: "environment"}});
            div.appendChild(video);

            imgElement = document.createElement('img');
            imgElement.style.position = 'absolute';
            imgElement.style.zIndex = 1;
            imgElement.onclick = () => { shutdown = true; };
            div.appendChild(imgElement);

            const instruction = document.createElement('div');
            instruction.innerHTML =
                '<span style="color: red; font-weight: bold;">' +
                'When finished, click here or on the video to stop this demo</span>';
            div.appendChild(instruction);
            instruction.onclick = () => { shutdown = true; };

            video.srcObject = stream;
            await video.play();

            captureCanvas = document.createElement('canvas');
            captureCanvas.width = 640; //video.videoWidth;
            captureCanvas.height = 480; //video.videoHeight;
            window.requestAnimationFrame(onAnimationFrame);

            return stream;
          }
          async function stream_frame(label, imgData) {
            if (shutdown) {
              removeDom();
              shutdown = false;
              return '';
            }

            var preCreate = Date.now();
            stream = await createDom();

            var preShow = Date.now();
            if (label != "") {
              labelElement.innerHTML = label;
            }

            if (imgData != "") {
              var videoRect = video.getClientRects()[0];
              imgElement.style.top = videoRect.top + "px";
              imgElement.style.left = videoRect.left + "px";
              imgElement.style.width = videoRect.width + "px";
              imgElement.style.height = videoRect.height + "px";
              imgElement.src = imgData;
            }

            var preCapture = Date.now();
            var result = await new Promise(function(resolve, reject) {
              pendingResolve = resolve;
            });
            shutdown = false;

            return {'create': preShow - preCreate,
                    'show': preCapture - preShow,
                    'capture': Date.now() - preCapture,
                    'img': result};
          }
        ''')
        display(js)

    def get_video_frame(self, label, bbox):
        return eval_js(f'stream_frame("{label}", "{bbox}")')

    def capture_samples(self, user_name='username', num_samples=100): # Caso queira trocar o nome do usuário, alterar aqui
        self.is_capturing = True
        self.should_stop = False
        os.makedirs(f'dataset/{user_name}', exist_ok=True)

        try:
            self.start_video_stream()
            count = 0
            bbox = ''

            while count < num_samples and not self.should_stop:
                js_reply = self.get_video_frame('Capturando...', bbox)
                if not js_reply:
                    break

                frame = self.js_to_image(js_reply["img"])
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = self.face_cascade.detectMultiScale(gray, 1.3, 5)

                for (x,y,w,h) in faces:
                    face_roi = cv2.resize(gray[y:y+h, x:x+w], (200, 200))
                    cv2.imwrite(f'dataset/{user_name}/face_{count}.jpg', face_roi)
                    count += 1
                    print(f'Amostras capturadas: {count}/{num_samples}')
                    time.sleep(0.3)

                # Verificação para parada manual
                if count >= num_samples:
                    print("Captura concluída!")
                    break

        finally:
            self.is_capturing = False
            display(HTML('<script>document.querySelector("button").click()</script>'))
            print("Webcam liberada")

    def train_model(self):
        self.recognizer = cv2.face.LBPHFaceRecognizer_create()
        faces = []
        labels = []
        label_ids = {}
        current_id = 0

        for root, dirs, _ in os.walk("dataset"):
            for dir_name in dirs:
                label_ids[dir_name] = current_id
                for file in os.listdir(f"{root}/{dir_name}"):
                    path = os.path.join(root, dir_name, file)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    faces.append(img)
                    labels.append(current_id)
                current_id += 1

        self.recognizer.train(faces, np.array(labels))
        self.labels = {v:k for k,v in label_ids.items()}
        self.recognizer.save("face_recognizer.yml")
        print("Modelo treinado com sucesso!")

    def run_recognition(self):
      if not self.recognizer:
          if os.path.exists("face_recognizer.yml"):
              self.recognizer = cv2.face.LBPHFaceRecognizer_create()
              self.recognizer.read("face_recognizer.yml")
          else:
              raise Exception("Modelo não treinado!")

      try:
          self.start_video_stream()
          bbox = ''

          while True:
              js_reply = self.get_video_frame('Reconhecendo...', bbox)
              if not js_reply:
                  break

              # Processar frame
              frame = self.js_to_image(js_reply["img"])
              gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
              faces = self.face_cascade.detectMultiScale(gray, 1.3, 5)

              # Criar overlay
              overlay = frame.copy()
              for (x,y,w,h) in faces:
                  roi_gray = gray[y:y+h, x:x+w]
                  label_id, conf = self.recognizer.predict(roi_gray)
                  name = self.labels.get(label_id, "Desconhecido")
                  color = (0,255,0) if conf < 70 else (0,0,255)

                  cv2.rectangle(overlay, (x,y), (x+w,y+h), color, 2)
                  cv2.putText(overlay, f"{name} ({conf:.1f})",
                            (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

              # Codificar para JPEG
              _, buffer = cv2.imencode('.jpg', overlay)
              bbox = 'data:image/jpg;base64,' + b64encode(buffer).decode('utf-8')

      finally:
            display(HTML('<script>document.querySelector("button").click()</script>'))
            print("Reconhecimento encerrado")

In [72]:
# Uso:
system = FaceRecognitionSystem()

In [73]:
# 1. Capturar amostras (executar separadamente)
system.capture_samples(num_samples=50)  # Clique na webcam para parar antes do término

<IPython.core.display.Javascript object>

Amostras capturadas: 1/50
Amostras capturadas: 2/50
Amostras capturadas: 3/50
Amostras capturadas: 4/50
Amostras capturadas: 5/50
Amostras capturadas: 6/50
Amostras capturadas: 7/50
Amostras capturadas: 8/50
Amostras capturadas: 9/50
Amostras capturadas: 10/50
Amostras capturadas: 11/50
Amostras capturadas: 12/50
Amostras capturadas: 13/50
Amostras capturadas: 14/50
Amostras capturadas: 15/50
Amostras capturadas: 16/50
Amostras capturadas: 17/50
Amostras capturadas: 18/50
Amostras capturadas: 19/50
Amostras capturadas: 20/50
Amostras capturadas: 21/50
Amostras capturadas: 22/50
Amostras capturadas: 23/50
Amostras capturadas: 24/50
Amostras capturadas: 25/50
Amostras capturadas: 26/50
Amostras capturadas: 27/50
Amostras capturadas: 28/50
Amostras capturadas: 29/50
Amostras capturadas: 30/50
Amostras capturadas: 31/50
Amostras capturadas: 32/50
Amostras capturadas: 33/50
Amostras capturadas: 34/50
Amostras capturadas: 35/50
Amostras capturadas: 36/50
Amostras capturadas: 37/50
Amostras c

Webcam liberada


In [74]:
# 2. Treinar (executar após captura)
system.train_model()

Modelo treinado com sucesso!


In [75]:
# 3. Reconhecer (executar após treino)
system.run_recognition()  # Clique na webcam para encerrar

<IPython.core.display.Javascript object>

Reconhecimento encerrado
